# Personal Finance Analysis

In [54]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import hashlib

%matplotlib inline

## Reading in the Data

Here, we expect some consistent format from the download button within Fidelity. 
In the event that we download more files for more up to date reflections on spending, I want to be able to easily add them and use the same code.
Here, we read in multiple csv files and provide unique hashes to reference them again in the future if we need to.

If we assume the average life of a human is 80 years x 365 days = 29,200 days, and we upper bound our spending by saying we have 100 transactions a day, we will have 2,920,000 transactions in a lifetime.

`log_36(2920000) = 4.166`, so if we round up to a hash of 5, we should have more than enough numbers to track all of our spending.

In [55]:

def read_transaction_csv():
    # Get list of CSV files
    csv_files = glob.glob('../data/transactions_*.csv')

    # If no files found, return empty DataFrame
    if not csv_files:
        return pd.DataFrame()

    # Read each file and add source column
    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df['SourceFile'] = file.split('/')[-1]  # Just the filename, not full path
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")

    # Combine all DataFrames
    if not dfs:
        return pd.DataFrame()

    return pd.concat(dfs, ignore_index=True)

def create_row_hashes(row):
    # Get column names to exclude
    exclude_cols = ['SourceFile']

    # Get row values for both intra and inter file hashes
    row_values_intra = [str(row[col]) for col in row.index if col not in exclude_cols]
    row_values_inter = [str(row[col]) for col in row.index]

    # Create hash strings
    row_str_intra = ''.join(row_values_intra)
    row_str_inter = ''.join(row_values_inter)

    # Create hashes
    intra_hash = hashlib.sha256(row_str_intra.encode()).hexdigest()
    inter_hash = hashlib.sha256(row_str_inter.encode()).hexdigest()

    return pd.Series([intra_hash[:5], inter_hash[:5]])

def add_row_hashes(df):
    if df.empty:
        return df

    # Apply to each row and get both hash values
    df[['IntraGroupingKey', 'InterGroupingKey']] = df.apply(create_row_hashes, axis=1)

    # Avoid duplicates with count as suffix
    df['IntraRowHashID'] = df.groupby('IntraGroupingKey').cumcount().add(1).astype(str).radd(df['IntraGroupingKey'] + '_')
    df['InterRowHashID'] = df.groupby('InterGroupingKey').cumcount().add(1).astype(str).radd(df['InterGroupingKey'] + '_')

    df.set_index('InterRowHashID', inplace=True)
    return df

# Preview data
add_row_hashes(read_transaction_csv())

,Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,SourceFile,IntraGroupingKey,InterGroupingKey,IntraRowHashID
InterRowHashID,,,,,,,,,,,,
4d236_1,6/11/2025,ALLY BANK DES TRANSFER INDN LEN G HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Transfers,No,No,($400),transactions_dummy_to_test_duplicates.csv,e3637,4d236,e3637_1
381b5_1,6/11/2025,VENMO DES PAYMENT INDN LEN HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Online Payment,No,No,($2500),transactions_dummy_to_test_duplicates.csv,94c5e,381b5,94c5e_1
8e2b1_1,6/11/2025,ALLY BANK DES ID LEN G HUANG INDN LEN G HUANG ...,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Income,No,No,$5000,transactions_dummy_to_test_duplicates.csv,72b02,8e2b1,72b02_1
5e116_1,6/10/2025,OZZIES FRESH MARKET NY,Bank of America - via Direct Web API,Customized Cash Rewards Visa Signature - 0966,Groceries,No,No,($6.99),transactions_dummy_to_test_duplicates.csv,4871c,5e116,4871c_1
30ac5_1,6/10/2025,SECURITY DEPOSIT UNION AVE,Venmo - via Direct Web Access,Venmo,Income,No,No,($2500),transactions_dummy_to_test_duplicates.csv,a1c2b,30ac5,a1c2b_1
...,...,...,...,...,...,...,...,...,...,...,...,...
a7b69_1,1/2/2024,ONLINE BANKING TRANSFER FROM CHK CONFIRMATION,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Transfers,No,No,$300,transactions_2025_06_13.csv,2d51e,a7b69,2d51e_2
05463_1,1/1/2024,COURT SQUARE DINER,Chase,Other Spending,Restaurants/Dining,No,No,($77.65),transactions_2025_06_13.csv,92d6d,05463,92d6d_2
2cf10_1,1/1/2024,NJT RAIL MYTIX,Chase,Other Spending,Public Transport,No,No,($8.5),transactions_2025_06_13.csv,ee82d,2cf10,ee82d_2


Now we can query for specific fields as such, in case we want to add logic overrides to specific rows.

In [83]:
# def find_row_from_hash(df, hash='f35f8'):
#     return df[df.index.str.startswith(hash)]

def find_row_from_hash(df, hash='f35f8'):
    idx_str = df.index.astype(str)
    hits = df[idx_str.str.startswith(str(hash))]

    if hits.empty:
        print(f"No rows match prefix {hash!r}")
    return hits

find_row_from_hash(add_row_hashes(read_transaction_csv()))

,Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,SourceFile,IntraGroupingKey,InterGroupingKey,IntraRowHashID
InterRowHashID,,,,,,,,,,,,
f35f8_1,6/11/2025,ALLY BANK DES TRANSFER INDN LEN G HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Transfers,No,No,($400),transactions_2025_06_13.csv,e3637,f35f8,e3637_2


# Dedupe the Data

We'll also be able to detect duplicates across files potentially like this. Specifically, we'll look for duplicates WITHIN files.

The `IntraGroupingkey` tells us about duplicates within a file (bank integration errors) and the `InterGroupingKey` tells us about duplicates across files (downloading csv errors).

In [57]:
def find_duplicate_groups():
    df = add_row_hashes(read_transaction_csv())
    return df[df.duplicated('IntraGroupingKey', keep=False)]

find_duplicate_groups().head()

,Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,SourceFile,IntraGroupingKey,InterGroupingKey,IntraRowHashID
InterRowHashID,,,,,,,,,,,,
4d236_1,6/11/2025,ALLY BANK DES TRANSFER INDN LEN G HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Transfers,No,No,($400),transactions_dummy_to_test_duplicates.csv,e3637,4d236,e3637_1
381b5_1,6/11/2025,VENMO DES PAYMENT INDN LEN HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Online Payment,No,No,($2500),transactions_dummy_to_test_duplicates.csv,94c5e,381b5,94c5e_1
8e2b1_1,6/11/2025,ALLY BANK DES ID LEN G HUANG INDN LEN G HUANG ...,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Income,No,No,$5000,transactions_dummy_to_test_duplicates.csv,72b02,8e2b1,72b02_1
5e116_1,6/10/2025,OZZIES FRESH MARKET NY,Bank of America - via Direct Web API,Customized Cash Rewards Visa Signature - 0966,Groceries,No,No,($6.99),transactions_dummy_to_test_duplicates.csv,4871c,5e116,4871c_1
30ac5_1,6/10/2025,SECURITY DEPOSIT UNION AVE,Venmo - via Direct Web Access,Venmo,Income,No,No,($2500),transactions_dummy_to_test_duplicates.csv,a1c2b,30ac5,a1c2b_1


But it looks like these are actually not duplicates, they are venmo requests of the same kind to different people! Let's understand this more.

In [ ]:
def duplicates_summary(df, key='IntraGroupingKey'):
    # Group and aggregate
    grouped = df.groupby(key).agg({
        'Description': 'first',
        'Date': ['min', 'max', 'count'],
        'Amount': 'first',
        'SourceFile': lambda x: ', '.join(set(x))  # Unique source files
    })

    # df.set_index('RowHashID', inplace=True)

    # Flatten multi-index columns
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

    # Rename columns
    grouped.rename(columns={
        'GroupingKey_': 'GroupID',
        'Description_first': 'Description',
        'Date_count': 'DuplicateCount',
        'Date_min': 'FirstDate',
        'Date_max': 'LastDate',
        'Amount_first': 'Amount',
        'SourceFile_<lambda>': 'SourceFiles'
    }, inplace=True)

    # Filter for actual duplicates (count > 1)
    duplicates = grouped[grouped['DuplicateCount'] > 1]

    # Sort by most frequent duplicates first
    return duplicates.sort_values('DuplicateCount', ascending=False)

duplicates_summary(add_row_hashes(read_transaction_csv()))

,Description,FirstDate,LastDate,DuplicateCount,Amount,SourceFiles
IntraGroupingKey,,,,,,
839bd,WILL PENN TAVERN,4/14/2024,4/14/2024,8,$26,"transactions_dummy_to_test_duplicates.csv, tra..."
fed3c,MESSED UP SOME MATH WILL PENN PT,4/14/2024,4/14/2024,8,$1.46,"transactions_dummy_to_test_duplicates.csv, tra..."
28855,RAJ S FRIDAY DINNER,4/21/2024,4/21/2024,8,$21.71,"transactions_dummy_to_test_duplicates.csv, tra..."
c685a,LUNCH,2/19/2024,2/19/2024,6,$20,"transactions_dummy_to_test_duplicates.csv, tra..."
9075a,BKOFAMERICA ATM WITHDRWL LAGUARDIA AIRPORT LAG...,1/21/2025,1/21/2025,6,($300),"transactions_dummy_to_test_duplicates.csv, tra..."
...,...,...,...,...,...,...
52bc0,REQUESTED TRANSFER TO LEN G HUANG BANK OF AMER...,1/29/2024,1/29/2024,2,($2120),"transactions_dummy_to_test_duplicates.csv, tra..."
52b46,SEAMLSS MEILAIWAHBAKER,11/9/2024,11/9/2024,2,($43.03),"transactions_dummy_to_test_duplicates.csv, tra..."
5295a,TST LOS TACOS NO,5/15/2024,5/15/2024,2,($15.7),"transactions_dummy_to_test_duplicates.csv, tra..."


How might we solve duplicate within a file though? Many of these are Venmo statements that some of the naming/description gets cut out from the description. Namely, should we add better descriptions before passing in the data? I think it'll be ok to coalesce all the `Intra` duplicates and then filter out any `Inter` duplicates AFTER coalescing all the `Intra` ones.

## Reformatting the Data

Here, we want to convert some of the values to things that will be easier to work with.
When reading it in, everything is a string. We want:
- the string `Date` to be datetime objects
- `Is Hidden` / `Is Pending` to bools
- `Amount` to be positive/negative numbers rather than accounting style strings.

In [59]:
def convert_types(df):
    # Convert dates to date times
    df['Date'] = pd.to_datetime(df['Date'])

    # Convert strings to bools
    df['Is Hidden'] = df['Is Hidden'].replace({'yes': True, 'no': False})
    df['Is Pending'] = df['Is Pending'].replace({'yes': True, 'no': False})

    # Remove $ and whitespace, remove parantheses from numbers, then convert to float
    df['Amount'] = df['Amount'].replace(r'[$\s]', '', regex=True).replace(r'\(([\d.]+)\)', r'-\1', regex=True).astype(float)

    return df

convert_types(add_row_hashes(read_transaction_csv()))

,Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,SourceFile,IntraGroupingKey,InterGroupingKey,IntraRowHashID
InterRowHashID,,,,,,,,,,,,
4d236_1,2025-06-11,ALLY BANK DES TRANSFER INDN LEN G HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Transfers,No,No,-400.00,transactions_dummy_to_test_duplicates.csv,e3637,4d236,e3637_1
381b5_1,2025-06-11,VENMO DES PAYMENT INDN LEN HUANG CO WEB,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Online Payment,No,No,-2500.00,transactions_dummy_to_test_duplicates.csv,94c5e,381b5,94c5e_1
8e2b1_1,2025-06-11,ALLY BANK DES ID LEN G HUANG INDN LEN G HUANG ...,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Income,No,No,5000.00,transactions_dummy_to_test_duplicates.csv,72b02,8e2b1,72b02_1
5e116_1,2025-06-10,OZZIES FRESH MARKET NY,Bank of America - via Direct Web API,Customized Cash Rewards Visa Signature - 0966,Groceries,No,No,-6.99,transactions_dummy_to_test_duplicates.csv,4871c,5e116,4871c_1
30ac5_1,2025-06-10,SECURITY DEPOSIT UNION AVE,Venmo - via Direct Web Access,Venmo,Income,No,No,-2500.00,transactions_dummy_to_test_duplicates.csv,a1c2b,30ac5,a1c2b_1
...,...,...,...,...,...,...,...,...,...,...,...,...
a7b69_1,2024-01-02,ONLINE BANKING TRANSFER FROM CHK CONFIRMATION,Bank of America - via Direct Web API,Adv Plus Banking - 2082,Transfers,No,No,300.00,transactions_2025_06_13.csv,2d51e,a7b69,2d51e_2
05463_1,2024-01-01,COURT SQUARE DINER,Chase,Other Spending,Restaurants/Dining,No,No,-77.65,transactions_2025_06_13.csv,92d6d,05463,92d6d_2
2cf10_1,2024-01-01,NJT RAIL MYTIX,Chase,Other Spending,Public Transport,No,No,-8.50,transactions_2025_06_13.csv,ee82d,2cf10,ee82d_2


# Coalescing Duplicates

Below, we seek to take any duplicates from `IntraGroupingKey` and consolidate them into one value with the sum of the amounts.

In [ ]:
def coalesce_duplicates(df, key='IntraGroupingKey'):
    if key not in df.columns:
        raise KeyError(f"{key!r} column not found. Columns: {df.columns.tolist()}")
    if df[key].isna().all():
        raise ValueError(f"All values in {key!r} are NaN/blank.")

    amount_cols = df.select_dtypes('number').columns.difference([key])
    other_cols  = df.columns.difference(amount_cols.union([key]))

    return (
        df.groupby(key, as_index=False, dropna=False)
          .agg({**{c: 'sum' for c in amount_cols},
                **{c: 'first' for c in other_cols}})
    )

coalesce_duplicates(convert_types(add_row_hashes(read_transaction_csv())))

,IntraGroupingKey,Amount,Account,Category,Date,Description,Institution,InterGroupingKey,IntraRowHashID,Is Hidden,Is Pending,SourceFile
0,00016,-10.86,Other Spending,Fast Food & Convenience,2025-01-03,TOO GOOD TO GO INC,Chase,0b879,00016_1,No,No,transactions_dummy_to_test_duplicates.csv
1,004af,36.66,Venmo,Rideshare,2025-01-24,AIRPORT TAXI S THERE BACK JUST ONE WAY FOR SACHIT,Venmo - via Direct Web Access,77992,004af_1,No,No,transactions_dummy_to_test_duplicates.csv
2,005a2,-800.00,Adv Plus Banking - 2082,Transfers,2024-05-01,ALLY BANK DES TRANSFER INDN LEN G HUANG CO WEB,Bank of America - via Direct Web API,47e7b,005a2_1,No,No,transactions_dummy_to_test_duplicates.csv
3,00725,-49.48,CREDIT CARD,Groceries,2024-03-11,TRADER JOES,Chase,09e01,00725_1,No,No,transactions_dummy_to_test_duplicates.csv
4,0083a,-375.64,Other Spending,Credit Card Payment,2024-11-03,FSI CONED BILL PAYMENT,Chase,daaa9,0083a_1,No,No,transactions_dummy_to_test_duplicates.csv
...,...,...,...,...,...,...,...,...,...,...,...,...
1941,fed3c,11.68,Venmo,Income,2024-04-14,MESSED UP SOME MATH WILL PENN PT,Venmo - via Direct Web Access,886c8,fed3c_1,No,No,transactions_dummy_to_test_duplicates.csv
1942,ff7de,184.96,Venmo,Income,2025-05-07,MAY CRUNCH,Venmo - via Direct Web Access,41f3e,ff7de_1,No,No,transactions_dummy_to_test_duplicates.csv
1943,ff972,58.40,Ally Savings Account,Interest Income,2025-06-08,INTEREST PAID,Ally,f5055,ff972_1,No,No,transactions_dummy_to_test_duplicates.csv
1944,ffa5a,147.70,Venmo,Income,2024-02-05,PICKLEBACK,Venmo - via Direct Web Access,fdd43,ffa5a_1,No,No,transactions_dummy_to_test_duplicates.csv


We can now se that there's no duplicates in our data! At least, no INTRA duplicates. What about inter?

In [80]:
duplicates_summary(coalesce_duplicates(convert_types(add_row_hashes(read_transaction_csv())), key='IntraGroupingKey'), key='InterGroupingKey')

,Description,FirstDate,LastDate,DuplicateCount,Amount,SourceFiles
InterGroupingKey,,,,,,


In [81]:
duplicates_summary(coalesce_duplicates(convert_types(add_row_hashes(read_transaction_csv())), key='InterGroupingKey'), key='IntraGroupingKey')

,Description,FirstDate,LastDate,DuplicateCount,Amount,SourceFiles
IntraGroupingKey,,,,,,
00016,TOO GOOD TO GO INC,2025-01-03,2025-01-03,2,-5.43,"transactions_dummy_to_test_duplicates.csv, tra..."
a6eb7,SMALL BUT HOPE THIS HELPS WITH SOMETHING BOOTH...,2025-04-02,2025-04-02,2,-100.00,"transactions_dummy_to_test_duplicates.csv, tra..."
aaa61,SEP OCT ELECTRICITY,2024-10-10,2024-10-10,2,64.85,"transactions_dummy_to_test_duplicates.csv, tra..."
aa877,FID BKG SVC LLC MONEYLINE,2024-07-02,2024-07-02,2,-700.00,"transactions_dummy_to_test_duplicates.csv, tra..."
aa4c0,LYFT RIDE,2024-04-16,2024-04-16,2,-49.03,"transactions_dummy_to_test_duplicates.csv, tra..."
...,...,...,...,...,...,...
52330,UBER HOME MIRAGE,2024-07-20,2024-07-20,2,-9.00,"transactions_dummy_to_test_duplicates.csv, tra..."
522fe,TRADER JOES,2024-09-15,2024-09-15,2,-48.71,"transactions_dummy_to_test_duplicates.csv, tra..."
520a4,PIZZERIA SCIALLA,2024-03-04,2024-03-04,2,-21.15,"transactions_dummy_to_test_duplicates.csv, tra..."


We can see depending on how we use this, we have cleaned up some of the duplicates! Though I think as I'm writing this I may have confused my notion of Intra and Inter lol. Since Inter takes in the source name, it will group things within a file. Then I want to remove duplicates based on Intra. Let me see if I can get some Inter's to show up together.

In [84]:
def test_coalesce_duplicates():
    df = convert_types(add_row_hashes(read_transaction_csv()))
    df = coalesce_duplicates(df, key='InterGroupingKey')
    row1 = find_row_from_hash(df, hash='886c8') # IntraGroupKey hash
    row2 = find_row_from_hash(df, hash='fed3c') # InterGroupKey hash
    return row1, row2

test_coalesce_duplicates()

No rows match prefix '886c8'
No rows match prefix 'fed3c'


(Empty DataFrame
 Columns: [InterGroupingKey, Amount, Account, Category, Date, Description, Institution, IntraGroupingKey, IntraRowHashID, Is Hidden, Is Pending, SourceFile]
 Index: [],
 Empty DataFrame
 Columns: [InterGroupingKey, Amount, Account, Category, Date, Description, Institution, IntraGroupingKey, IntraRowHashID, Is Hidden, Is Pending, SourceFile]
 Index: [])

Hmmm... something's not adding up